In [1]:
import torch
import random
import evaluate

from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import DataLoader
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification
from transformers import RobertaModel, RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer

c:\Users\incognito\miniconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
peft_model_name = 'roberta-base-peft'
modified_base = 'roberta-base-modified'
base_model = 'roberta-base'

In [4]:
dataset = load_dataset('ag_news')
tokenizer = RobertaTokenizer.from_pretrained(base_model)

c:\Users\incognito\miniconda3\envs\llm\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
len(dataset['train'])

120000

In [6]:
dataset['train'][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

In [7]:

def preprocess(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)
    
    
tokenized_dataset = dataset.map(preprocess, batched=True,  remove_columns=["text"])
train_dataset=tokenized_dataset['train']

# divide the total test data into two equal sets using shard
test_dataset=tokenized_dataset['test'].shard(num_shards=2, index=0)
eval_dataset=tokenized_dataset['test'].shard(num_shards=2, index=1)

In [8]:
print(dataset['train'].features['label']) 
# look at the print to find about the right key.
class_labels = dataset['train'].features['label'].names
num_labels = len(class_labels)
print(f"number of labels: {num_labels}")
print(f"the labels: {class_labels}")

ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)
number of labels: 4
the labels: ['World', 'Sports', 'Business', 'Sci/Tech']


In [9]:
# Create an (id:label) mapping for clasisfing
id2label = {i: label for i, label in enumerate(class_labels)}
print(id2label)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
print(data_collator)

{0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}
DataCollatorWithPadding(tokenizer=RobertaTokenizer(name_or_path='roberta-base', vocab_size=50265, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}, padding=True, m

In [10]:
# Trainign arguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    learning_rate=5e-5,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    fp16=True if torch.cuda.is_available() else False

)

def get_trainer(model, training_args, train_dataset):
      return  Trainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          data_collator=data_collator,
      )

c:\Users\incognito\miniconda3\envs\llm\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# model = AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label)
# model.to(device)
# fine_tune_trainer = get_trainer(model, training_args, train_dataset)

# fine_tune_trainer.train()

In [12]:
# PEFT Training
# model_ft = AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label)

# peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)
# peft_model = get_peft_model(model_ft, peft_config)
# peft_model.to(device)
# print("PEFT MODEL")
# peft_model.print_trainable_parameters()
# peft_lora_finetuning_trainer = get_trainer(peft_model, training_args, train_dataset)
# peft_lora_finetuning_trainer.train()

In [13]:
# Save
# tokenizer.save_pretrained(modified_base)
# peft_model.save_pretrained(peft_model_name)

In [17]:
# Testing(Inference) of the LORA fine tune model.

peft_model = AutoPeftModelForSequenceClassification.from_pretrained(peft_model_name, id2label=id2label, local_files_only=True)
peft_model.to(device)
tokenizer = AutoTokenizer.from_pretrained(modified_base, local_files_only=True)

def classify(text, deivce = device):
    inputs = tokenizer(text,truncation=True, padding=True, return_tensors='pt')
    inputs.to(device)
    output = peft_model(**inputs)
    
    prediction = output.logits.argmax(dim=-1).item()
    
    print(f'\n Class: {prediction}, Label: {id2label[prediction]}, Text: {text}')
    

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
classify( "Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his ...")
classify( "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.")


 Class: 1, Label: Sports, Text: Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his ...

 Class: 2, Label: Business, Text: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindlinand of ultra-cynics, are seeing green again.


In [19]:
metric = evaluate.load('accuracy')

def evaluate_model(model, dataset):
    
    eval_dataloader = DataLoader(dataset.rename_column("label", "labels"), batch_size=8, collate_fn=data_collator)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device)
    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=1)
        references = batch['labels']
        metric.add_batch(
            predictions=predictions,
            references=references
        )
    
    eval_metric = metric.compute()
    print(eval_metric)

In [20]:
# Base model evaluation on test data
evaluate_model(AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label), test_dataset)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 475/475 [00:26<00:00, 18.18it/s]

{'accuracy': 0.2513157894736842}


In [21]:
# LORA (peft) model evaluation on test data
evaluate_model(peft_model, test_dataset)

100%|██████████| 475/475 [00:27<00:00, 17.12it/s]

{'accuracy': 0.9171052631578948}


In [22]:
# Fully Fine tuned model evaluation on test data
# evaluate_model(fine_tune_trainer.model, test_dataset)

In [23]:
peft_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleD